In [14]:
import random
import numpy as np
import pandas as pd
import time
import selenium
from selenium import webdriver
import typing
from typing import List, Dict

In [15]:
class ElizaScraper():
    def __init__(self):
        self.first_page = 'https://immoelissa.be/immobilier/?sort=prix-c'
        self.driver = webdriver.Firefox()
        self.data = []
        self.all_advert_urls = []
        
    
    def scrape(self):
        """
        Scrape methods calls other methods in the class in order. TODO add threading support.
        """
        
        self.readPagination()
        
        for url in self.all_advert_urls:
            result = self.readAdPage(url)
            self.data.append(result)
    
    def getAdUrl(self, inp_url):
        """
        Method that gathers urls for each individual ad
        
        :inp_url: a url that links to a page containing several urls of individual ads
        """
        self.driver.get(inp_url)
        url = []
        
        time.sleep(random.uniform(1.0, 2.0))
        for elem in self.driver.find_elements_by_tag_name("a"):
            link = elem.get_attribute("href")
            if not link is None and "annonces" in link:
                url.append(link)
        return url
    
    def readPagination(self):
        """
        readPagination gathers all pages that contain independent ads. The first one is hardcoded and 
        aferwards it formats the URL as a string until no new ads are found.

        :return: a list of URLs that link to each page of a search query. 
        """

        # initiaze a list for storing all the page URLS
        page_urls = []
        page_urls.append(self.first_page)

        # initialize a list for storing all the seperate links to individual ads.
        advert_urls = []
        counter = 2

        # get all the links for the first page
        advert_urls = self.getAdUrl(self.first_page)
        self.all_advert_urls.extend(advert_urls)

        # keep iterating over page numbers until no more links are found.
        while advert_urls != []:
            url2 = f"https://immoelissa.be/immobilier/?pg={counter}&sort=prix-c"
            advert_urls = self.getAdUrl(url2)
            self.all_advert_urls.extend(advert_urls)
            counter += 1
        
        return self.all_advert_urls
    
    def readAdPage(self, inp_url):
        """
        Method the reads all the information on the page and writes it to a dict using the key, 
        value pairs used on the website in question. 

        :inp_url: get the input URL that we will be gathering information from.
        :return: a dict that stores scraped data
        """

        # initialize some variables.
        detailsKeys = []
        detailsValues = []
        details = {}

        # grab the URL page code
        self.driver.get(inp_url)
        time.sleep(random.uniform(0.4, 0.8))

        # find the tables in the page and split data in 2 types, keys and values.
        for desc_list in self.driver.find_elements_by_tag_name("dt"):
            detail = desc_list.text.split("\n")
            detailsKeys.extend(detail)

        for desc_list in self.driver.find_elements_by_tag_name("dd"):
            detail = desc_list.text.split("\n")
            detailsValues.extend(detail)

        # build a dict containing all gathered data
        for idx, x in enumerate(detailsKeys):
            details[x] = detailsValues[idx]

        return details


In [16]:
scraper = ElizaScraper()
scraper.scrape()

KeyboardInterrupt: 